# Hypothetical Document Embedding (HyDE) for RAG
 - 실제 문서를 벡터화하는 대신, **질문을 기반으로 LLM이 생성한 가상의 문서(Hypothetical Document)를 임베딩하여 검색**하는 기법.

 - 질문 → 가상 문서 → 임베딩 → 벡터DB에서 유사 문서 검색 → 정답 추론

 - HyDE 사용 이유?
    - 임베딩된 질문이 벡터 공간에서 문서와 잘 매칭되지 않을 수 있음
    - LLM이 생성한 문서는 더 "정보 중심적"이므로 검색 성능이 개선됨

In [ ]:
%pip install -qU langchain-community tiktoken faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 26.1 MB/s eta 0:00:00


In [ ]:
!wget https://raw.githubusercontent.com/FingerPointLab/llm-lecture-materials/refs/heads/main/manufacturing_industry_docs.txt

--2025-03-24 11:58:30--  https://raw.githubusercontent.com/FingerPointLab/llm-lecture-materials/refs/heads/main/manufacturing_industry_docs.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2920 (2.9K) [text/plain]
Saving to: ‘manufacturing_industry_docs.txt.1’

manufacturing_indus 100%[===================>]   2.85K  --.-KB/s    in 0s      

2025-03-24 11:58:30 (39.7 MB/s) - ‘manufacturing_industry_docs.txt.1’ saved [2920/2920]



In [ ]:
from langchain.document_loaders import TextLoader
from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 🔹 OpenAI API 키 설정 (환경변수 또는 직접 입력)
import os
os.environ["OPENAI_API_KEY"] = "openai-api-key"

In [ ]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [ ]:
# 1. 데이터 로딩
loader = TextLoader('manufacturing_industry_docs.txt')
documents = loader.load()

# 2. 텍스트 분할 (Chunking)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10, separators=["\n\n"])
docs = text_splitter.split_documents(documents)

docs

[Document(metadata={'source': 'manufacturing_industry_docs.txt'}, page_content='[문서1]\n스마트 팩토리는 제조 공정의 자동화와 데이터 기반 최적화를 통해 생산성을 향상시키는 시스템이다.'),
 Document(metadata={'source': 'manufacturing_industry_docs.txt'}, page_content='[문서2]\n반도체 제조 공정에는 웨이퍼 세정, 포토리소그래피, 식각, 증착, 검사 공정이 포함된다.'),
 Document(metadata={'source': 'manufacturing_industry_docs.txt'}, page_content='[문서3]\n디지털 트윈은 실제 공장의 물리적 상태를 가상 공간에서 시뮬레이션할 수 있게 해준다.'),
 Document(metadata={'source': 'manufacturing_industry_docs.txt'}, page_content='[문서4]\nAI 기반 품질 검사는 제품의 외관이나 결함을 자동으로 탐지하고 분류하는 데 사용된다.'),
 Document(metadata={'source': 'manufacturing_industry_docs.txt'}, page_content='[문서5]\n산업용 IoT는 센서와 장비를 인터넷으로 연결해 공정 데이터를 실시간 수집하고 분석한다.'),
 Document(metadata={'source': 'manufacturing_industry_docs.txt'}, page_content='[문서6]\n3D 프린팅 기술은 제품 개발 주기를 단축하고 맞춤형 생산을 가능하게 만든다.'),
 Document(metadata={'source': 'manufacturing_industry_docs.txt'}, page_content='[문서7]\nMES(Manufacturing Execution System)는 생산 현장의 작업 지시, 모니터링, 품질 관리를 통합적으로 수행하는 시스템

In [ ]:
# 1. 임베딩 및 검색기 세팅
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)

# 2. 사용자 질문
query = "반도체 제조 공정은 어떤 공정으로 이루어져 있나요?"
# query = "스마트 팩토리와 디지털 트윈의 차이는?"
# query = "디지털 트윈은 스마트 팩토리에서 어떤 역할을 하나요?"
# query = "디지털 전환이 제조업에 미치는 영향은?"

print("📝 기본 Retrieval:\n")
retriever = vectorstore.as_retriever()
pretty_print_docs(retriever.invoke(query))

📝 기본 Retrieval:

Document 1:

[문서16]
반도체 제조는 고도로 정밀한 공정으로 구성되어 있으며, 수십 개의 공정을 거쳐 최종 제품이 생산된다.
----------------------------------------------------------------------------------------------------
Document 2:

[문서2]
반도체 제조 공정에는 웨이퍼 세정, 포토리소그래피, 식각, 증착, 검사 공정이 포함된다.
----------------------------------------------------------------------------------------------------
Document 3:

[문서17]
반도체 제조 공정에서 포토리소그래피는 반도체 회로를 웨이퍼 표면에 패턴화하는 핵심 공정이다.
----------------------------------------------------------------------------------------------------
Document 4:



[문서18]
반도체 제조 공정 중간 및 최종 단계에서 광학 검사 장비나 전자빔 스캐닝 장비를 활용하여 불량 여부를 검사하고, 수집된 데이터를 통해 공정 조건을 실시간으로 피드백한다.


In [ ]:

# HyDE 문서 생성 (LLM을 활용한 가상 문서 생성)
llm = ChatOpenAI(temperature=0.7)
hypothetical_doc = llm.predict(f"다음 질문에 답하기 위해 필요한 문서를 작성하세요: {query}")

print("📝 HyDE 문서:\n", hypothetical_doc)

# 4. HyDE 문서 임베딩 → 검색
# retriever = FAISS.from_texts([hypothetical_doc], embedding_model)
retrieved_docs = vectorstore.similarity_search(hypothetical_doc, k=3)

print("\n🔍HyDE 유사 문서 Top 3:")
# for i, doc in enumerate(retrieved_docs):
#     print(f"{i+1}. {doc.page_content}")
pretty_print_docs(retrieved_docs)


📝 HyDE 문서:
 반도체 제조 공정은 일반적으로 다음과 같은 단계로 이루어져 있습니다:

1. 웨이퍼 제조: 웨이퍼는 반도체 칩의 기본적인 기판으로 사용되는 실리콘 원판입니다. 웨이퍼 제조 단계에서는 산화나 불순물 제거 등의 과정을 통해 웨이퍼 표면을 깨끗하게 만들고 원하는 두께로 다듬어줍니다.

2. 노광: 노광은 반도체 칩에 패턴을 만들기 위해 사용되는 공정으로, 미세한 빛을 이용해 반도체 칩의 회로를 형성하는 과정입니다.

3. 식각: 식각은 노광으로 만들어진 패턴을 따라 반도체 칩의 층을 조절하는 과정입니다. 이 단계에서는 필요한 물질만 남기고 나머지를 제거하여 반도체 칩의 회로를 형성합니다.

4. 증착: 증착은 반도체 칩의 층을 증가시키는 과정으로, 다양한 물질을 증착하여 반도체 칩의 성능을 향상시키는 역할을 합니다.

5. 응고: 응고는 반도체 칩을 냉각하여 안정화시키고 마무리 단계로, 반도체 칩의 최종적인 형태를 만들어줍니다.

이러한 단계를 거쳐 반도체 제조 공정이 완료되며, 최종적으로 테스트를 거쳐 제품의 품질을 확인합니다.

🔍HyDE 유사 문서 Top 3:
Document 1:

[문서2]
반도체 제조 공정에는 웨이퍼 세정, 포토리소그래피, 식각, 증착, 검사 공정이 포함된다.
----------------------------------------------------------------------------------------------------
Document 2:

[문서16]
반도체 제조는 고도로 정밀한 공정으로 구성되어 있으며, 수십 개의 공정을 거쳐 최종 제품이 생산된다.
----------------------------------------------------------------------------------------------------
Document 3:

[문서17]
반도체 제조 공정에서 포토리소그래피는 반도체 회로를 웨이퍼 표면에 패턴화하는 핵심 공정이다.
